# MyAnimeList

### Prerequisites:

In [ ]:
%pip install --upgrade pyspark
%pip install --upgrade numpy
%pip install --upgrade pandas
%pip freeze > requirements.txt


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
spark = SparkSession\
    .builder\
    .getOrCreate()

anime_df = spark.read.csv(
    "./csv/AnimeList.csv",
    header=True,
    inferSchema=True,
    sep=",",
)

In [ ]:
from pyspark.sql.functions import split, regexp_replace, col
from pyspark.sql.types import FloatType

anime_df = anime_df\
    .drop(
        "title",
        "title_english",
        "title_japanese",
        "title_synonyms",
        "image_url",
        "background",
        "broadcast",
        "related",
        "opening_theme",
        "ending_theme",
        "studio",
        "premiere",
        "producer",
        "licensor",
        "rank",
    )\
    .fillna({"genre": ""})\
    .withColumn(
        "genre",
        split(
            regexp_replace('genre', ' ', ''),
            ",",
        )
    )

for col_name in [
    "episodes",
    "score",
    "scored_by",
    "popularity",
    "members",
    "favorites",
    ]:
    anime_df = anime_df.withColumn(col_name, col(col_name).cast(FloatType()))

In [ ]:
from pyspark.ml.feature import CountVectorizer

anime_df = CountVectorizer(
    inputCol="genre",
    outputCol="genre_fv"
)\
    .fit(anime_df)\
    .transform(anime_df)\
    .drop("genre")

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

categoricalColumns = ["type", "source", "status", "rating"]
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(
        inputCol=categoricalCol,
        outputCol=categoricalCol + "Index",
    )
    encoder = OneHotEncoder(
        inputCol=stringIndexer.getOutputCol(),
        outputCol=categoricalCol + "classVec",
    )
    stages += [stringIndexer, encoder]

In [8]:
from pyspark.ml.feature import VectorAssembler

numericCols = ["episodes", "score", "scored_by", "popularity", "members", "favorites", "genre_fv"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = 

['typeclassVec',
 'sourceclassVec',
 'statusclassVec',
 'ratingclassVec',
 'episodes',
 'score',
 'scored_by',
 'popularity',
 'members',
 'favorites',
 'genre_fv']